## AMM Optimization, 3rd model (time series)

Now, we want to model the problem of splitting a trade, across time with stochastic variation of price.

We don't take trading fees into account, for simplicity.

### 1. Problem formulation

Idea:
Make x,y change according to some distribution in time (so price changes acc to some distribution) and then we would have a multi-period-stochastic optimization?

Let’s say binomial model, price goes up 5% with probability p or down 5% with 1-p every time period pre trade. xy is cte. y/x= p so we have the liquidity. 

Then if we trade, we move the market to post trade. Then next period post-trade up or down 5% again.

Finally, minimize sum expected_values_t


So we define $x_t$ and $y_t$ as the amount of tokens in the market at time $t$. We assume that the price of token $x$ is $p_t = \frac{y_t}{x_t}$

If we fix a time t and a pool i.

Let $p_{t}$ be the price of token $x$ at time $t$.

we have the following constraints:

\begin{align}
x_{i,t}*y_{i,t} = k_{i} \\
p_{t} = \frac{y_{i,t}}{x_{i,t}}

\end{align}

So we can derive the value of the pool at time $t$ as:

\begin{align}
y_{i,t} = \sqrt{k_{i}p_{t}} \\
x_{i,t} = \sqrt{\frac{k_{i}}{p_{t}}}
\end{align}

As before, the price i'll pay for that transaction will be:

\begin{align}
\frac{y_{i,t}}{x_{i,t} + \Delta x_{i,t}}
\end{align}

and the new price of the pool will be:

\begin{align}
q_{t} = \frac{y_{i,t} + \Delta y_{i,t}}{x_{i,t} - \Delta x_{i,t}}
\end{align}

Given the binomial variation, the expected value of the pool at time $t+1$ will be:

\begin{align}
E[p_{t+1}] = 1.05q_{t}p + 0.95q_{t}(1-p)
\end{align}
Where p is the probability of the price going up.


No we will use those considerations to define the optimization problem.

I want to minimize the expected value of the price I will pay for the transaction of size order_amount, given the binomial variation of the price.

We define the variables as:
- $\Delta x_{i,t}$: split of the trade at time $t$ in pool i
- $p_{t}$: price of token $x$ at beginning of period $t$ (expected value)
- $q_{t}$: price of token $x$ at the end of period $t$ 

My objective function to minimize is:

\begin{align}
\sum_{t=0}^{T} \sum_{i=0}^{N} \frac{y_{i,t}}{x_{i,t} + \Delta x_{i,t}}
\end{align}

Which can be rewritten as using the expectation and the transfer theorem:

\begin{align}
\sum_{t=0}^{T} \sum_{i=0}^{N} \frac{\sqrt{1.05k_{i}q_{t-1}}}{\sqrt{\frac{k_{i}}{1.05q_{t-1}}} + \Delta x_{i,t}}p + \frac{\sqrt{0.95k_{i}q_{t-1}}}{\sqrt{\frac{k_{i}}{0.95q_{t-1}}} + \Delta x_{i,t}}(1-p)
\end{align}

and the constraints are:

\begin{align}
\sum_{i=0}^{N} \Delta x_{i,t} &= order\_amount \\
q_t &= \frac{y_{i,t} + \Delta y_{i,t}}{x_{i,t} - \Delta x_{i,t}} \\
E[p_{t+1}] &= 1.05q_{t}p + 0.95q_{t}(1-p)
\end{align}

This is false check how can be corrected